In [1]:
import numpy as np
from pathlib import Path
import time
import matplotlib.pyplot as plt
start_time = time.time()

import xobjects as xo
import xtrack as xt
import xpart as xp
import xobjects as xo
import xcoll as xc

import apertls
from matplotlib.colors import LogNorm

In [2]:
# This script takes around 8 minutes on a modern CPU (80s preparation+interpolation, 400s tracking)
beam = 1
plane = 'DNeg'#'DPpos'

num_particles  = 5000
sweep          = 3500
sweep          = -abs(sweep) if plane == 'DPpos' else abs(sweep)
#num_turns      = int(20*abs(sweep))
num_turns=6000

In [3]:
nemitt_x = 3.5e-6
nemitt_y = 3.5e-6

In [4]:
line = xt.Line.from_json('../injection_lines/sps_with_aperture_inj_q20_beam_sagitta.json')

#setting all cavities to 200 (even 800 cavities) because rf sweep not implemented
cavity_elements, cavity_names = line.get_elements_of_type(xt.Cavity)
for name in cavity_names:
    line[name].frequency = 200e6
    line[name].lag = 180

line['acl.31735'].voltage = 0


Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           


In [5]:
line.get_s_position(['bpcn.12508.a_aper','bpcn.12508.b_aper'])

[np.float64(799.5902), np.float64(799.825)]

In [32]:
799.825 - 799.5902

0.23480000000006385

In [16]:
X0 = 1/(0.2857142857142857/1.757 + 0.7142857142857143/3.039e4)

In [17]:
X0

6.148611294007604

In [15]:
5/7

0.7142857142857143

In [18]:
1/((2/7)/13.84 + (5/7)/36.62)

24.90694748666105

In [30]:
X_Fe = 1.757 #cm
X_C = 19.32 
X_air = 3.039e4
X_steel = (0.98/X_Fe + 0.02/X_C)**(-1)
X0 = (5/7 / X_air + 2/7 / X_steel)**(-1)

In [31]:
X0

6.262453505056473

In [27]:
X_steel

1.789535836328612

In [29]:
X0

35.75071782483751